# Network Operations

In this notebook we will assemble our project.  We will explore different functions on our dataset and compile them into a workflow ready for production.

The functions we will use will be a mix of `hub` based functions from our [MLRun Functions](http://github.com/mlrun/functions) repo, local and git based notebooks.

> The notebook should be run after generating the data in the [Generator Notebook](generator.ipynb)


we will start by setting up our environment, Loading MLRun and some utilities we will need

In [1]:
# Utils
import os
import json
import urllib
import numpy as np

# MLRun imports
from mlrun import mlconf

# Setup API Endpoint
mlconf.dbpath = 'http://mlrun-api:8080'

In [2]:
# Set Hub URL address if using a local version
# mlconf.hub_url = '/User/functions/{name}/function.yaml'

Now lets define our current project 

## Create a project from a git repository

In [3]:
from mlrun import new_project

# update the dir and repo to reflect real locations 
# the remote git repo must be initialized in GitHub
project_dir = os.path.abspath('./')
remote_git = 'https://github.com/mlrun/demo-network-operations.git'

# Create the project
newproj = new_project('network-operations', project_dir, init_git=True)

# We can update our project directory to the latest status by running
# newproj.pull()

Now that we have our project directory, lets forword our artifacts there to keep track of them

In [4]:
# Define an artifact path to keep track of where our artifacts are going
ARTIFACT_PATH =  os.path.join(os.path.abspath(newproj.context), 'artifacts')
mlconf.artifact_path = ARTIFACT_PATH

## Create and run functions

As we receive a new dataset, the first thing we would like to do is to explore it a bit, we can do that using our `describe` function in `mlrun/functions`

In [5]:
from mlrun import mount_v3io, new_model_server

In [6]:
# Import the functions
# Functions From hub
newproj.set_function(func='hub://aggregate', name='aggregate')
newproj.set_function(func='hub://describe', name='describe')
newproj.set_function(func='hub://feature_selection', name="feature_selection")
newproj.set_function(func='hub://sklearn_classifier', name='train')
newproj.set_function(func='hub://test_classifier', name='test')
newproj.set_function(func='hub://model_server_tester', name="model_server-tester")
newproj.set_function(func='hub://concept_drift', name="concept_drift")
newproj.set_function(func='hub://stream_to_parquet', name="s2p")
newproj.set_function(func='hub://virtual_drift', name="virtual_drift")

# Streaming
src_path = os.path.abspath('src/')
newproj.set_function(func=os.path.join(src_path, 'generator.yaml'), name='generator')
newproj.set_function(func=os.path.join(src_path, 'preprocessor.yaml'), name='preprocessor')
newproj.set_function(func=os.path.join(src_path, 'inference-server.yaml'), name="serving")
newproj.set_function(func=os.path.join(src_path, 'labeled_stream_creator.yaml'), name="labeled_stream")

## Generate the dataset
If needed go to [Generator](./generator.ipynb) and run the local workflow to generate the metrics dataset to `data/metrics`

## Run the functions locally to develop the workflow

now we can **Run** the function locally on our sample data, we would like to get some details on our `raw` data

## Register raw data as project level artifact

In [7]:
# Define base Dataset
import random
data_dir = os.path.join(os.path.abspath(newproj.context), 'data')
dataset_filename = random.choice(list(filter(lambda x: x.endswith('parquet'), os.listdir(data_dir))))
metrics_path = os.path.join(data_dir, dataset_filename)

import pandas as pd
# Drop alternate error columns
label_column = 'is_error'
raw = pd.read_parquet(metrics_path)
raw = raw.drop([col for col in raw.columns if (col != label_column) & (col.endswith(label_column))], axis=1)

# Add to the project as a Dataset Artifact
from mlrun.artifacts import DatasetArtifact
metrics_artifact = DatasetArtifact(key='metrics', df=raw, format='parquet', target_path=os.path.join(data_dir, 'metrics.pq'))
# newproj.log_artifact(metrics_artifact)
newproj.log_artifact('metrics', target_path='/User/demo-network-operations/data/metrics.pq')

### Get statistics about the metrics data

In [8]:
from mlrun import NewTask, mount_v3io

In [9]:
describe_task = NewTask(
    name="describe", 
    handler="summarize",  
    params={"key": "summary", 
            "label_column": label_column, 
            'class_labels': ['0', '1'],
            'plot_hist': True,
            'plot_dest': 'plots-metrics'},
    inputs={"table": '/User/demo-network-operations/data/metrics.pq'},
    artifact_path=ARTIFACT_PATH)

In [10]:
decsribe_run = newproj.func('describe').apply(mount_v3io()).run(describe_task)

[mlrun] 2020-07-01 08:31:26,632 starting run describe uid=7b52930d82c145a989ac9e82dbcabab8  -> http://mlrun-api:8080
[mlrun] 2020-07-01 08:31:26,745 Job is running in the background, pod: describe-vgm4f
[mlrun] 2020-07-01 08:31:37,381 run executed, status=completed
/opt/conda/lib/python3.7/site-packages/seaborn/distributions.py:288: UserWarning: Data must have variance to compute a kernel density estimate.
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/seaborn/distributions.py:288: UserWarning: Data must have variance to compute a kernel density estimate.
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/seaborn/distributions.py:288: UserWarning: Data must have variance to compute a kernel density estimate.
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/seaborn/distributions.py:288: UserWarning: Data must have variance to compute a kernel density estimate.
  warnings.warn(msg, UserWarning)
final state: succeeded

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
network-operations,...dbcabab8,0,Jul 01 08:31:32,completed,describe,v3io_user=adminkind=jobowner=adminhost=describe-vgm4f,table,"key=summarylabel_column=is_errorclass_labels=['0', '1']plot_hist=Trueplot_dest=plots-metrics",,histogramsimbalanceimbalance-weights-veccorrelation-matrixcorrelation


to track results use .show() or .logs() or in CLI: 
!mlrun get run 7b52930d82c145a989ac9e82dbcabab8 --project network-operations , !mlrun logs 7b52930d82c145a989ac9e82dbcabab8 --project network-operations
[mlrun] 2020-07-01 08:31:45,952 run executed, status=completed


### Create the feature vector

We will use our [Aggregate](https://github.com/mlrun/functions/blob/master/aggregate/aggregate.ipynb) function to create rolling window features for our feature vector.

In doing so we hope that we could help our algorithms identify local errors by using a windowed trend

In [9]:
# Define aggregate task
from mlrun import NewTask
aggregate_task = NewTask(
    name='aggregate',
    params={'metrics': ['cpu_utilization', 'throughput', 'packet_loss', 'latency'],
            'metric_aggs': ['mean', 'sum', 'std', 'var', 'min', 'max', 'median'],
            'suffix': 'daily',
            'append_to_df': True,
            'window': 20,
            'center': False,
            'save_to': os.path.join('data', 'aggregate.pq'),
            'drop_na': True},
    inputs={'df_artifact': '/User/demo-network-operations/data/metrics.pq'},
    handler='aggregate')

In [10]:
newproj.func('aggregate').spec.image = 'mlrun/ml-models:unstable-py36'
newproj.func('aggregate').spec.imagePullPolicy = 'Always'
aggregate_run = newproj.func('aggregate').apply(mount_v3io()).run(aggregate_task)

[mlrun] 2020-07-05 15:43:10,364 starting run aggregate uid=fd3e17bd40fd4cb78bb7c70cdd420225  -> http://mlrun-api:8080
[mlrun] 2020-07-05 15:43:10,487 Job is running in the background, pod: aggregate-vrfjz
[mlrun] 2020-07-05 15:43:15,109 Aggregating /User/demo-network-operations/data/metrics.pq
[mlrun] 2020-07-05 15:43:15,265 Logging artifact

[mlrun] 2020-07-05 15:43:15,504 run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
network-operations,...dd420225,0,Jul 05 15:43:15,completed,aggregate,v3io_user=adminkind=jobowner=adminhost=aggregate-vrfjz,df_artifact,"metrics=['cpu_utilization', 'throughput', 'packet_loss', 'latency']metric_aggs=['mean', 'sum', 'std', 'var', 'min', 'max', 'median']suffix=dailyappend_to_df=Truewindow=20center=Falsesave_to=data/aggregate.pqdrop_na=True",,aggregate


to track results use .show() or .logs() or in CLI: 
!mlrun get run fd3e17bd40fd4cb78bb7c70cdd420225 --project network-operations , !mlrun logs fd3e17bd40fd4cb78bb7c70cdd420225 --project network-operations
[mlrun] 2020-07-05 15:43:19,782 run executed, status=completed


### Get statistics about the feature vector

In [84]:
aggregate_describe_task = NewTask(
    name="describe-aggregate", 
    handler="summarize",  
    params={"key": "summary", 
            "label_column": label_column, 
            'class_labels': ['0', '1'],
            'plot_hist': True,
            'plot_dest': 'plots-aggregate',
            'sample': 0.3},
    inputs={"table": aggregate_run.outputs['aggregate']},
    artifact_path=ARTIFACT_PATH)

In [ ]:
aggregate_decsribe_run = newproj.func('describe').apply(mount_v3io()).run(aggregate_describe_task)

## Create workflow to train the model
After reviewing the data and creating the feature vector we move to training our model.  
For this task we will use an **LGBM** classifier.  To control the training process we will supply a `model_config` dictionary with the following parameters:
- **CLASS**: Model-specific parameters.
- **FIT**: Training parameters (like epoch when needed)
- **META**: Model and Package version

### Setup model configurations

In [8]:
model_configs = {
    "CLASS" : {
        "boosting_type"      : "gbdt",
        "num_leaves"         : 300,
        "max_depth"          : 50,
        "learning_rate"      : 0.1,
        "n_estimators"       : 300,
        "objective"          : "binary",
        "scale_pos_weight"   : 1,    
        "min_split_gain"     : 0.0,
        "min_child_samples"  : 20,
        "subsample"          : 1,
        "colsample_bytree"   : 1,
        "reg_alpha"          : 0,
        "reg_lambda"         : 1,
        "n_jobs"             : 16,
        "silent"             : True,
        "importance_type"    : "split",
        "random_state"       : 1},
    "FIT" : {
        "verbose"               : False
    },
    "META" : {
        "class" : "lightgbm.sklearn.LGBMClassifier",
        "version" : "2.3.1"
    }
}
model_config_path = os.path.join(os.path.abspath(newproj.context), 'data', 'lgb_model.json')
with open(model_config_path, 'w') as f:
          f.write(json.dumps(model_configs))

In [9]:
newproj.log_artifact('lgb_configs',
                     target_path = os.path.abspath(model_config_path))

### Create Pipeline Workflow

In [8]:
%%writefile src/workflow.py
from kfp import dsl
from mlrun import mount_v3io, mlconf
import os
from nuclio.triggers import V3IOStreamTrigger

funcs = {}
projdir = os.getcwd()
label_column = 'is_error'
model_inference_stream = '/users/admin/demo-network-operations/streaming/inference_stream'
model_inference_url = f'http://v3io-webapi:8081{model_inference_stream}'


def init_functions(functions: dict, project=None, secrets=None):
    tags = {'ml-models': 'unstable-py36',
            'mlrun': 'unstable'}
    for f in functions.values():
        # Add V3IO Mount
        f.apply(mount_v3io())
        if hasattr(f.spec, 'image'):
            if not f.spec.image.endswith('unstable-py36') and (not f.spec.image == ''):
                current_image = f.spec.image.split(':')[0]
                fn_image = f"{current_image}:{tags[current_image.split('/')[1]]}"
                print(fn_image)
                f.spec.image = fn_image
        
        # Always pull images to keep updates
        f.spec.image_pull_policy = 'Always'
        
    functions['serving'].metadata.name = 'netops-server'
    functions['serving'].spec.min_replicas = 1
    functions['serving'].spec.build.baseImage = 'mlrun/mlrun:unstable'
    
    # Define inference-stream related triggers
    functions['serving'].set_envs({'INFERENCE_STREAM': model_inference_stream})
    functions['s2p'].add_trigger('labeled_stream', V3IOStreamTrigger(url=f'{model_inference_url}@s2p'))
    functions['s2p'].set_envs({'window': 10,
                               'features': 'cpu_utilization',
                               'save_to': '/User/demo-network-operations/streaming/inference_pq/',
                               'base_dataset': '/User/demo-network-operations/artifacts/test_set_preds.parquet',
                               'hub_url': '/User/functions/{name}/function.yaml',
                               'mount_path': '~/',
                               'mount_remote': '/User'})
                
        
@dsl.pipeline(
    name='Network Operations Demo',
    description='Train a Failure Prediction LGBM Model over sensor data'
)
def kfpipeline(
        df_artifact = '/User/demo-network-operations/data/metrics.pq',
        metrics = ['cpu_utilization', 'throughput', 'packet_loss', 'latency'],
        metric_aggs = ['mean', 'sum', 'std', 'var', 'min', 'max', 'median'],
        suffix = 'daily',
        window = 20,
        describe_table = 'netops',
        describe_sample = 0.3,
        label_column = label_column,
        class_labels = [1, 0],
        SAMPLE_SIZE      = -1, # -n for random sample of n obs, -1 for entire dataset, +n for n consecutive rows
        TEST_SIZE        = 0.1,       # 10% set aside
        TRAIN_VAL_SPLIT  = 0.75,      # remainder split into train and val
        aggregate_fn_url = '/User/functions/aggregate/function.yaml',
        streaming_metrics_table = '/User/demo-network-operations/streaming/metrics',
        streaming_features_table = '/User/demo-network-operations/streaming/features',
        streaming_predictions_table = '/User/demo-network-operations/streaming/predictions',
        streaming_labeled_table = '/users/admin/demo-network-operations/streaming/labeled_stream',
        predictions_col = 'predictions',
        secs_to_generate = 10,
        deploy_streaming = True,
        deploy_concept_drift = True
    ):
    
    # Run preprocessing on the data
    aggregate = funcs['aggregate'].as_step(name='aggregate',
                                                  params={'metrics': metrics,
                                                          'metric_aggs': metric_aggs,
                                                          'suffix': suffix,
                                                          },
                                                  inputs={'df_artifact': df_artifact},
                                                  outputs=['aggregate'],
                                                  handler='aggregate',
                                                  image='mlrun/ml-models:unstable-py36')

    describe = funcs['describe'].as_step(name='describe-aggregation',
                                                handler="summarize",  
                                                params={"key": f"{describe_table}_aggregate", 
                                                        "label_column": label_column, 
                                                        'class_labels': class_labels,
                                                        'plot_hist': True,
                                                        'plot_dest': 'plots/aggregation',
                                                        'sample': describe_sample},
                                                inputs={"table": aggregate.outputs['aggregate']},
                                                outputs=["summary", "scale_pos_weight"])
    
    feature_selection = funcs['feature_selection'].as_step(name='feature_selection',
                                                           handler='feature_selection',
                                                           params={'k': 5,
                                                                   'min_votes': 3,
                                                                   'label_column': label_column},
                                                           inputs={'df_artifact': aggregate.outputs['aggregate']},
                                                           outputs=['feature_scores', 
                                                                    'max_scaled_scores_feature_scores'
                                                                    'selected_features_count', 
                                                                    'selected_features'],
                                                           image='mlrun/ml-models:unstable-py36')
    
    describe = funcs['describe'].as_step(name='describe-feature-vector',
                                            handler="summarize",  
                                            params={"key": f'{describe_table}_features', 
                                                    "label_column": label_column, 
                                                    'class_labels': class_labels,
                                                    'plot_hist': True,
                                                    'plot_dest': 'plots/feature_vector'},
                                            inputs={"table": feature_selection.outputs['selected_features']},
                                            outputs=["summary", "scale_pos_weight"])
    
    train = funcs['train'].as_step(name='train',
                                   params={"sample"          : SAMPLE_SIZE, 
                                           "label_column"    : label_column,
                                           "test_size"       : TEST_SIZE},
                                   inputs={"dataset"         : feature_selection.outputs['selected_features']},
                                   hyperparams={'model_pkg_class': ["sklearn.ensemble.RandomForestClassifier", 
                                                                    "sklearn.linear_model.LogisticRegression",
                                                                    "sklearn.ensemble.AdaBoostClassifier"]},
                                   selector='max.accuracy',
                                   outputs=['model', 'test_set'],
                                   image='mlrun/ml-models:unstable-py36')
    
    test = funcs['test'].as_step(name='test',
                                 handler='test_classifier',
                                 params={'label_column': label_column,
                                         'predictions_column': predictions_col},
                                 inputs={'models_path': train.outputs['model'],
                                         'test_set': train.outputs['test_set']},
                                 image='mlrun/ml-models:unstable-py36')
    
    # deploy the model using nuclio functions
    deploy = funcs['serving'].deploy_step(env={'model_path': train.outputs['model'],
                                               'FEATURES_TABLE': streaming_features_table,
                                               'PREDICTIONS_TABLE': streaming_predictions_table,
                                               'prediction_col': predictions_col}, 
                                          tag='v1')
    
    # test out new model server (via REST API calls)
    tester = funcs["model_server-tester"].as_step(name='model-tester',
                                                  params={'addr': deploy.outputs['endpoint'], 
                                                          'model': "predictor",
                                                          'label_column': label_column},
                                                  inputs={'table': train.outputs['test_set']})
    
    with dsl.Condition(deploy_streaming == True):
    
        # Streaming demo functions
        preprocessor = funcs['preprocessor'].deploy_step(env={ 'aggregate_fn_url': aggregate_fn_url,
                                                                'METRICS_TABLE': streaming_metrics_table,
                                                                'FEATURES_TABLE': streaming_features_table,
                                                                'metrics': metrics,
                                                                'metric_aggs': metric_aggs,
                                                                'suffix': suffix,
                                                                'base_dataset': '/User/demo-network-operations/artifacts/selected_features.parquet',
                                                                'label_col': label_column}).after(tester)

        labeled_stream_creator = funcs['labeled_stream'].deploy_step(env={'METRICS_TABLE': streaming_metrics_table,
                                                                                  'PREDICTIONS_TABLE': streaming_predictions_table,
                                                                                  'OUTPUT_STREAM': streaming_labeled_table,
                                                                                  'label_col': label_column,
                                                                                  'prediction_col': predictions_col}).after(tester)

        generator = funcs['generator'].deploy_step(env={'SAVE_TO': streaming_metrics_table,
                                                        'SECS_TO_GENERATE': secs_to_generate}).after(preprocessor)
        
        with dsl.Condition(deploy_concept_drift == True):

            concept_builder = funcs['concept_drift'].deploy_step(skip_deployed=True)

            concept_drift = funcs['concept_drift'].as_step(name='concept_drift_deployer',
                                                           params={'models': ['ddm', 'eddm', 'pagehinkley'],
                                                                   'label_col': label_column,
                                                                   'prediction_col': predictions_col,
                                                                   'hub_url': '/User/functions/{name}/function.yaml',
                                                                   'output_tsdb': '/User/network-operations/streaming/drift_tsdb',
                                                                   'input_stream': 'http://v3io-webapi:8081/users/admin/network-operations/streaming/labeled_stream@cd2',
                                                                   'output_stream': '/User/network-operations/streaming/drift_stream'},
                                                           inputs={'base_dataset': '/User/demo-network-operations/artifacts/test_set_preds.parquet'},
                                                           artifact_path=mlconf.artifact_path,
                                                           image=concept_builder.outputs['image']).after(labeled_stream_creator)

            s2p = funcs['s2p'].deploy_step(env={'window': 10,
                                                'features': 'cpu_utilization',
                                                'save_to': '/User/demo-network-operations/streaming/inference_pq/',
                                                'base_dataset': '/User/demo-network-operations/artifacts/test_set_preds.parquet',
                                                'hub_url': '/User/functions/{name}/function.yaml',
                                                'mount_path': '~/',
                                                'mount_remote': '/User'}).after(tester)
    

Overwriting src/workflow.py


## Add workflow

In [9]:
newproj.set_workflow('main', os.path.join(os.path.abspath(newproj.context), 'src', 'workflow.py'))

## Save Project

In [10]:
newproj.save(os.path.join(newproj.context, 'project.yaml'))

### Run workflow

In [11]:
newproj.run('main', artifact_path=ARTIFACT_PATH, dirty=True)

[mlrun] 2020-07-13 17:07:15,754 WARNING!, you seem to have uncommitted git changes, use .push()
mlrun/ml-models:unstable-py36
mlrun/ml-models:unstable-py36
mlrun/ml-models:unstable-py36
mlrun/ml-models:unstable-py36
mlrun/ml-models:unstable-py36
mlrun/mlrun:unstable
mlrun/ml-models:unstable-py36
[mlrun] 2020-07-13 17:07:16,208 using in-cluster config.


/conda/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "JsonArray" based on the value "['cpu_utilization', 'throughput', 'packet_loss', 'latency']".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/conda/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "JsonArray" based on the value "['mean', 'sum', 'std', 'var', 'min', 'max', 'median']".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/conda/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Integer" based on the value "20".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/conda/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Mi

[mlrun] 2020-07-13 17:07:17,430 Pipeline run id=5208dea0-4a26-4e20-b217-d8edb06251db, check UI or DB for progress


'5208dea0-4a26-4e20-b217-d8edb06251db'